In [72]:
#import package
import pandas as pd
from geopy.distance import geodesic

pd.set_option('display.max_columns', None)

In [73]:
#import data
merged = pd.read_csv('/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')
metro = pd.read_csv('/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/Metro_Stations_in_DC.csv')
night_club = pd.read_csv('/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/night_club.csv')

In [74]:
#smaller data set
night_club

,OBJECTID,NAME,ADDRESS,X,Y,ADDRID
0,1,RED LOUNGE,2013 14TH STREET NW,397258.37,138845.27,240339
1,2,ROCK & ROLL HOTEL,1353 H STREET NE,401163.56,136891.68,286092
2,3,ROSEBAR,1215 CONNECTICUT AVENUE NW,396433.43,137617.28,241766
3,4,ROYAL PALACE,1805 CONNECTICUT AVENUE NW,396027.19,138505.86,242756
4,5,SMOKE & BARREL,2471 18TH STREET NW,396336.03,139379.24,235595
...,...,...,...,...,...,...
57,58,Penthouse Pool Club,1612 U STREET NW,396736.10,138763.71,241860
58,59,POLICY,1904 14TH STREET NW,397208.43,138656.29,218552
59,60,POV,515 15TH STREET NW,397103.43,136567.81,279820
60,61,PURE LOUNGE,1326 U STREET NW,397334.26,138769.63,279068


In [75]:
#creating new column for tuple of x and y coordinates
merged['coords'] = tuple(zip(merged.LONGITUDE, merged.LATITUDE))
metro['coords2'] = tuple(zip(metro.X, metro.Y))
night_club['coords3'] = tuple(zip(night_club.X, night_club.Y))

In [76]:
#distance from metro
distance = []
for index, row in merged.iterrows():
    for index2, row2 in metro.iterrows():
        x = geodesic(row['coords'], row2['coords2']).miles
        z = [index, index2, x]
        distance.append(z)
df = pd.DataFrame(distance, columns = ['index', 'metro_stop_id', 'distance'])
df2 = df.groupby(['index'], as_index=False).agg(
{
    'distance':'min'
})
df2

,index,distance
0,0,1.127080
1,1,0.191281
2,2,0.083638
3,3,0.135380
4,4,0.084503
...,...,...
147380,147380,0.418831
147381,147381,0.718883
147382,147382,0.345018
147383,147383,0.442869


In [78]:
#join metro distance into crime data
df3 = pd.concat([merged, df2], axis=1)
df3

,Unnamed: 0,SHIFT,METHOD,OFFENSE,XBLOCK,YBLOCK,WARD,DISTRICT,LATITUDE,LONGITUDE,date,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8,coords,index,distance
0,0,MIDNIGHT,OTHERS,THEFT F/AUTO,403249.170000,138982.570000,5.0,5.0,38.918707,-76.962533,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,"(-76.96253311529999, 38.9187067412)",0,1.127080
1,1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,399214.790000,137753.750000,6.0,1.0,38.907643,-77.009053,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,"(-77.0090530223, 38.907642772399996)",1,0.191281
2,2,MIDNIGHT,OTHERS,THEFT F/AUTO,398532.800000,137052.350000,6.0,1.0,38.901323,-77.016914,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,"(-77.0169144784, 38.901323441100004)",2,0.083638
3,3,DAY,OTHERS,THEFT F/AUTO,398374.890000,135044.830000,6.0,1.0,38.883239,-77.018730,2017-10-27,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,"(-77.0187301806, 38.8832386792)",3,0.135380
4,4,DAY,OTHERS,THEFT/OTHER,400246.540000,139229.040000,5.0,5.0,38.920933,-76.997157,2017-10-28,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,"(-76.99715700510001, 38.9209329952)",4,0.084503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147380,147380,DAY,OTHERS,BURGLARY,402085.390000,138351.140000,5.0,5.0,38.913022,-76.975955,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"(-76.9759548322, 38.9130221422)",147380,0.418831
147381,147381,DAY,OTHERS,THEFT F/AUTO,402855.420321,134087.377978,7.0,6.0,38.874610,-76.967094,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,"(-76.9670938752, 38.874610442)",147381,0.718883
147382,147382,DAY,OTHERS,THEFT F/AUTO,400720.210000,142604.330000,5.0,4.0,38.951338,-76.991691,2021-02-03,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"(-76.99169130060001, 38.9513383536)",147382,0.345018
147383,147383,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,401535.270000,131621.840000,8.0,7.0,38.852403,-76.982313,2021-02-04,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"(-76.98231291260001, 38.852403147800004)",147383,0.442869


In [79]:
#dropping columns
del df3['Unnamed: 0']
del df3['date']
del df3['coords']
del df3['index']
df3

<bound method DataFrame.info of            SHIFT  METHOD                     OFFENSE         XBLOCK  \
0       MIDNIGHT  OTHERS                THEFT F/AUTO  403249.170000   
1       MIDNIGHT   KNIFE  ASSAULT W/DANGEROUS WEAPON  399214.790000   
2       MIDNIGHT  OTHERS                THEFT F/AUTO  398532.800000   
3            DAY  OTHERS                THEFT F/AUTO  398374.890000   
4            DAY  OTHERS                 THEFT/OTHER  400246.540000   
...          ...     ...                         ...            ...   
147380       DAY  OTHERS                    BURGLARY  402085.390000   
147381       DAY  OTHERS                THEFT F/AUTO  402855.420321   
147382       DAY  OTHERS                THEFT F/AUTO  400720.210000   
147383       DAY   KNIFE  ASSAULT W/DANGEROUS WEAPON  401535.270000   
147384       DAY  OTHERS                THEFT F/AUTO  403293.260929   

               YBLOCK  WARD  DISTRICT   LATITUDE  LONGITUDE  total_cases  \
0       138982.570000   5.0       5.0  

In [80]:

df3.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/crime_covid_metro_11_28_2021.csv')


,SHIFT,METHOD,OFFENSE,XBLOCK,YBLOCK,WARD,DISTRICT,LATITUDE,LONGITUDE,total_cases,daily_cases,daily_death,OUT_POSITIVE_QUARANTINE_PSMPD,MO,TU,WE,TH,FR,SA,SU,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ward_1,ward_2,ward_3,ward_4,ward_5,ward_6,ward_7,ward_8,distance
0,MIDNIGHT,OTHERS,THEFT F/AUTO,403249.170000,138982.570000,5.0,5.0,38.918707,-76.962533,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1.127080
1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,399214.790000,137753.750000,6.0,1.0,38.907643,-77.009053,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.191281
2,MIDNIGHT,OTHERS,THEFT F/AUTO,398532.800000,137052.350000,6.0,1.0,38.901323,-77.016914,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.083638
3,DAY,OTHERS,THEFT F/AUTO,398374.890000,135044.830000,6.0,1.0,38.883239,-77.018730,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0.135380
4,DAY,OTHERS,THEFT/OTHER,400246.540000,139229.040000,5.0,5.0,38.920933,-76.997157,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.084503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147380,DAY,OTHERS,BURGLARY,402085.390000,138351.140000,5.0,5.0,38.913022,-76.975955,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.418831
147381,DAY,OTHERS,THEFT F/AUTO,402855.420321,134087.377978,7.0,6.0,38.874610,-76.967094,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.718883
147382,DAY,OTHERS,THEFT F/AUTO,400720.210000,142604.330000,5.0,4.0,38.951338,-76.991691,37365.0,166.0,6.0,106.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.345018
147383,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,401535.270000,131621.840000,8.0,7.0,38.852403,-76.982313,37634.0,269.0,8.0,104.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.442869
